In [1]:
# Code3.ipynb
# from Simple.../Code1 but mugged
#
# 2021-09-01:  Make version to *archive* for future reference and use
#
# 2021-07-18:  Add "What day is today?"  Ans: *date* which is replaced by actual day
#              Changes to intents.json
#
# URL: https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44
import nltk
# commented out since already downloaded
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

2021-09-02 10:08:12.790871: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-02 10:08:12.790927: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents3.json').read()   # new named, added '3'
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

34 documents
10 classes ['date', 'doch', 'goodbye', 'greeting', 'ja', 'nein', 'options', 'quatch', 'stimmt', 'thanks']
53 unique lemmatized words ["'s", ',', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'doch', 'dock', 'doe', 'for', 'german', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'in', 'is', 'ja', 'later', 'me', 'mean', 'nein', 'next', 'nice', 'offered', 'provide', 'quatch', 'quatsch', 'see', 'stimmt', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'today', 'what', 'you']


In [6]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print(">>> Training data created")

>>> Training data created


/tmp/ipykernel_663/860445403.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print(">>> Model created")

2021-09-02 10:08:15.334816: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-02 10:08:15.335006: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-02 10:08:15.335131: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-964CGUG): /proc/driver/nvidia/version does not exist
2021-09-02 10:08:15.336007: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/bob/nlpiaVenv/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: U

Epoch 1/200
7/7 [==============================] - 0s 2ms/step - loss: 2.3585 - accuracy: 0.0588
Epoch 2/200
7/7 [==============================] - 0s 1ms/step - loss: 2.2515 - accuracy: 0.1471
Epoch 3/200
7/7 [==============================] - 0s 2ms/step - loss: 2.1256 - accuracy: 0.2941
Epoch 4/200
7/7 [==============================] - 0s 1ms/step - loss: 2.1859 - accuracy: 0.2353
Epoch 5/200
7/7 [==============================] - 0s 1ms/step - loss: 2.0018 - accuracy: 0.4118
Epoch 6/200
7/7 [==============================] - 0s 1ms/step - loss: 1.9958 - accuracy: 0.4118
Epoch 7/200
7/7 [==============================] - 0s 2ms/step - loss: 1.8894 - accuracy: 0.3824
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 1.8857 - accuracy: 0.4412
Epoch 9/200
7/7 [==============================] - 0s 1ms/step - loss: 1.7509 - accuracy: 0.3235
Epoch 10/200
7/7 [==============================] - 0s 1ms/step - loss: 1.6194 - accuracy: 0.5882
Epoch 11/200
7/7 [===========

7/7 [==============================] - 0s 1ms/step - loss: 0.0733 - accuracy: 1.0000
Epoch 85/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2187 - accuracy: 0.9118
Epoch 86/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0789 - accuracy: 0.9706
Epoch 87/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1249 - accuracy: 0.9706
Epoch 88/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2894 - accuracy: 0.9118
Epoch 89/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0404 - accuracy: 1.0000
Epoch 90/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2119 - accuracy: 0.9118
Epoch 91/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1931 - accuracy: 0.9118
Epoch 92/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0718 - accuracy: 1.0000
Epoch 93/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1423 - accuracy: 0.9706
Epoch 94/200
7/7 [===============

7/7 [==============================] - 0s 1ms/step - loss: 0.0455 - accuracy: 1.0000
Epoch 165/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 166/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 167/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0488 - accuracy: 1.0000
Epoch 168/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 169/200
7/7 [==============================] - 0s 911us/step - loss: 0.0612 - accuracy: 0.9706
Epoch 170/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0191 - accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 979us/step - loss: 0.0431 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0265 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0165 - accuracy: 1.0000
Epoch 174/200
7/7 [=